In [10]:
import time
from numba import jit, njit
import numba
import numpy as np
import heapq

In [2]:
def timer(func, *args):
    start = time.time()
    out = func(*args)
    print(f'time: {time.time()-start} sec. \noutput:\n{out}')

In [3]:
def ack(arr, n):
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            arr[i,j] = i*100 + j*10 + n
    return arr

In [4]:
arr = np.zeros([10000,10000], dtype=np.int64)
timer(ack, arr, 9)
np.sum(arr)

time: 14.716437578201294 sec. 
output:
[[      9      19      29 ...   99979   99989   99999]
 [    109     119     129 ...  100079  100089  100099]
 [    209     219     229 ...  100179  100189  100199]
 ...
 [ 999709  999719  999729 ... 1099679 1099689 1099699]
 [ 999809  999819  999829 ... 1099779 1099789 1099799]
 [ 999909  999919  999929 ... 1099879 1099889 1099899]]


54995400000000

In [5]:
from numba import f8, i8
@jit(i8[:,:](i8[:,:], i8),nopython=True)
def jit_ack(arr, n):
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            arr[i,j] = i*100 + j*10 + n
    return arr

In [6]:
arr = np.zeros([10000,10000], dtype=np.int64)
timer(jit_ack, arr, 9)
np.sum(arr)

time: 0.23197174072265625 sec. 
output:
[[      9      19      29 ...   99979   99989   99999]
 [    109     119     129 ...  100079  100089  100099]
 [    209     219     229 ...  100179  100189  100199]
 ...
 [ 999709  999719  999729 ... 1099679 1099689 1099699]
 [ 999809  999819  999829 ... 1099779 1099789 1099799]
 [ 999909  999919  999929 ... 1099879 1099889 1099899]]


54995400000000

In [12]:
def heap(q, arr):
    qq = list(q)
    heapq.heapify(qq)
    c = 0
    while len(qq) != 0:
        arr[c] = heapq.heappop(qq)
        c+=1

In [13]:
@jit('void(i8[:],i8[:])', nopython=True)
def heap_j(q, arr):
    qq = list(q)
    heapq.heapify(qq)
    c = 0
    while len(qq) != 0:
        arr[c] = heapq.heappop(qq)
        c+=1

In [14]:
from numba import prange, i8
@njit('i8[:](i8)')
def create_random(n):
    return np.array([np.random.randint(n) for _ in prange(n)], dtype=np.int64)

In [15]:
N = 10000000
l = create_random(N)
result = np.empty_like(l, dtype=np.int64)

In [17]:
timer(heap,l,result)

time: 17.24725842475891 sec. 
output:
None


In [16]:
timer(heap_j,l,result)

time: 2.5699656009674072 sec. 
output:
None


In [55]:
result

array([      1,       2,       2, ..., 9999994, 9999997, 9999998])

In [62]:
from numba import f4, i8, void
@njit(void(i8[:,:], i8[:,:], i8))
def create_pool_j(output, table, r):

    for i in range(len(table)):
        up = np.max(np.array([0, i-r]))
        down = np.min(np.array([len(table)-1, i+r]))
        for j in range(len(table[0])):
            left = np.max(np.array([0, j-r]))
            right = np.min(np.array([len(table[0])-1, j+r]))
            output[i,j] = np.sum(table[up:down+1, left:right+1])

            # s = 0
            # for k in range(up, down+1):
            #     for l in range(left, right+1):
            #         s += table[k,l]
            # output[i,j] = s
    


In [54]:
def create_pool(output, table, r):

    for i in range(len(table)):
        up = max([0, i-r])
        down = min([len(table)-1, i+r])
        for j in range(len(table[0])):
            left = max([0, j-r])
            right = min([len(table[0])-1, j+r])
            output[i,j] = np.sum(table[up:down+1, left:right+1])

In [66]:
m = 5000
r = 1
table = np.arange(m**2).reshape((m,m))
output = np.empty_like(table)


In [58]:
create_pool(output, table,r)

In [59]:
create_pool_j(output, table, r)

In [67]:
for i in range(len(table)):
    for j in range(len(table[1])):
        s = len(table)+len(table[0])

In [68]:
l1 = len(table)
l2 = len(table[0])
for i in range(len(table)):
    for j in range(len(table[1])):
        s = l1+l2